# 🐳 Docker Services Management - GenAI

**Module :** 00-GenAI-Environment  
**Niveau :** 🟢 Débutant  
**Technologies :** Docker, Docker Compose, Services GenAI  
**Durée estimée :** 45 minutes  

## 🎯 Objectifs d'Apprentissage

- [ ] Comprendre l'architecture Docker GenAI
- [ ] Gérer les services locaux (ComfyUI, Ollama, etc.)
- [ ] Diagnostiquer et résoudre les problèmes Docker
- [ ] Optimiser les performances des conteneurs

## 📚 Prérequis

- Docker Desktop installé et configuré
- Environnement GenAI validé (notebook 00-1)
- Compréhension basique de Docker

## 🏗️ Architecture Services

```
┌─────────────────────────────────────────────┐
│              GenAI Docker Stack             │
├─────────────────────────────────────────────┤
│ 🎨 ComfyUI (8188)     │ 🤖 Ollama (11434)  │
│ 📊 Jupyter (8888)     │ 🗄️  PostgreSQL     │
│ 🔄 Redis Cache        │ 📈 Monitoring      │
└─────────────────────────────────────────────┘
```

In [ ]:
# Paramètres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"  # "interactive" ou "batch"
skip_widgets = False           # True pour mode batch MCP
debug_level = "INFO"           

# Configuration Docker
docker_compose_file = "docker-compose.genai.yml"
service_timeout = 60           # Timeout services en secondes
health_check_retries = 3       # Nombre de tentatives health check

# Services à gérer
managed_services = [
    "comfyui",        # Interface génération images
    "ollama",         # LLM local
    "jupyter",        # Notebooks interactifs
    "redis",          # Cache rapide
    "postgres"        # Base de données
]

# Options de diagnostic
show_logs = True               # Afficher les logs services
show_resources = True          # Monitorer ressources CPU/RAM
auto_restart = False           # Redémarrage automatique si échec

In [ ]:
# Setup environnement et imports
import os
import sys
import subprocess
import json
import time
import psutil
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Optional, Any
import yaml
import logging

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging, load_genai_config
        print("✅ Helpers GenAI importés")
    except ImportError:
        print("⚠️  Helpers GenAI non disponibles")

# Configuration logging
logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('docker_services')

# Détection Docker
try:
    subprocess.run(['docker', '--version'], check=True, capture_output=True)
    DOCKER_AVAILABLE = True
    print("✅ Docker CLI détecté")
except (subprocess.CalledProcessError, FileNotFoundError):
    DOCKER_AVAILABLE = False
    print("❌ Docker CLI non trouvé")

print(f"🐳 Docker Services Management")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Services gérés: {', '.join(managed_services)}")

In [ ]:
# Classe DockerServiceManager
class DockerServiceManager:
    """Gestionnaire des services Docker GenAI"""
    
    def __init__(self, compose_file: str = "docker-compose.genai.yml"):
        self.compose_file = Path(compose_file)
        self.services_status = {}
        self.logger = logging.getLogger('docker_manager')
    
    def run_command(self, cmd: List[str], timeout: int = 30) -> Dict[str, Any]:
        """Exécute une commande système avec gestion d'erreur"""
        try:
            result = subprocess.run(
                cmd, capture_output=True, text=True, timeout=timeout
            )
            return {
                'success': result.returncode == 0,
                'stdout': result.stdout,
                'stderr': result.stderr,
                'returncode': result.returncode
            }
        except subprocess.TimeoutExpired:
            return {
                'success': False,
                'error': f'Timeout après {timeout}s',
                'stdout': '', 'stderr': ''
            }
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'stdout': '', 'stderr': ''
            }
    
    def get_docker_info(self) -> Dict[str, Any]:
        """Informations système Docker"""
        info = {'docker_available': DOCKER_AVAILABLE}
        
        if DOCKER_AVAILABLE:
            # Version Docker
            version_result = self.run_command(['docker', '--version'])
            if version_result['success']:
                info['docker_version'] = version_result['stdout'].strip()
            
            # Docker Compose
            compose_result = self.run_command(['docker', 'compose', 'version'])
            info['docker_compose_available'] = compose_result['success']
            if compose_result['success']:
                info['docker_compose_version'] = compose_result['stdout'].strip()
            
            # Espace disque
            system_result = self.run_command(['docker', 'system', 'df'])
            if system_result['success']:
                info['disk_usage'] = system_result['stdout']
        
        return info
    
    def list_services(self) -> Dict[str, Any]:
        """Liste tous les services Docker en cours"""
        if not DOCKER_AVAILABLE:
            return {'error': 'Docker non disponible'}
        
        # Services via docker ps
        ps_result = self.run_command(['docker', 'ps', '--format', 'json'])
        services = []
        
        if ps_result['success'] and ps_result['stdout'].strip():
            for line in ps_result['stdout'].strip().split('\n'):
                try:
                    container = json.loads(line)
                    services.append({
                        'name': container.get('Names', ''),
                        'image': container.get('Image', ''),
                        'status': container.get('Status', ''),
                        'ports': container.get('Ports', '')
                    })
                except json.JSONDecodeError:
                    continue
        
        return {'services': services, 'count': len(services)}

# Initialisation manager
docker_manager = DockerServiceManager(docker_compose_file)
print(f"🔧 DockerServiceManager initialisé")

### Presentation du DockerServiceManager

La classe `DockerServiceManager` encapsule les operations de gestion des conteneurs Docker.

**Methodes principales** :

| Methode | Description | Retour |
|---------|-------------|--------|
| `run_command(cmd, timeout)` | Execute une commande Docker avec timeout | Dict avec success/stdout/stderr |
| `get_docker_info()` | Retourne les infos systeme Docker | Dict avec versions et espace disque |
| `list_services()` | Liste les conteneurs Docker actifs | Dict avec liste des services |

**Utilisation typique** :
```python
# Lister les services actifs
services = docker_manager.list_services()
for service in services['services']:
    print(f"{service['name']}: {service['status']}")
```

> **Note technique** : Le manager utilise `subprocess.run()` avec capture_output pour executer les commandes Docker de manière sécurisée et récupérer les sorties standard et d'erreur.

In [ ]:
# Diagnostic Docker et services
print("\n" + "="*50)
print("🐳 DIAGNOSTIC DOCKER SERVICES")
print("="*50)

# Informations Docker système
docker_info = docker_manager.get_docker_info()
print(f"\n📋 INFORMATIONS SYSTÈME:")
for key, value in docker_info.items():
    status_icon = "✅" if (key.endswith('_available') and value) or (key.endswith('_version') and value) else "❌" if key.endswith('_available') else "ℹ️"
    print(f"{status_icon} {key.replace('_', ' ').title()}: {value}")

# Liste des services actifs
if DOCKER_AVAILABLE:
    print(f"\n🔍 SERVICES DOCKER ACTIFS:")
    services_info = docker_manager.list_services()
    
    if 'services' in services_info and services_info['services']:
        for service in services_info['services']:
            print(f"  🟢 {service['name']}")
            print(f"     📦 Image: {service['image']}")
            print(f"     📊 Status: {service['status']}")
            if service['ports']:
                print(f"     🌐 Ports: {service['ports']}")
            print()
    else:
        print("  📝 Aucun service Docker actif")
    
    # Ressources système si demandé
    if show_resources:
        print(f"\n💻 RESSOURCES SYSTÈME:")
        cpu_percent = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory()
        disk = psutil.disk_usage('/')
        
        print(f"  🧠 CPU: {cpu_percent}%")
        print(f"  💾 RAM: {memory.percent}% ({memory.used // (1024**3):.1f}GB / {memory.total // (1024**3):.1f}GB)")
        print(f"  💿 Disque: {disk.percent}% ({disk.used // (1024**3):.1f}GB / {disk.total // (1024**3):.1f}GB)")
else:
    print("\n❌ Docker non disponible - Diagnostic limité")

# Test connectivité services GenAI communs
print(f"\n🔗 TEST CONNECTIVITÉ SERVICES:")
common_ports = {
    'ComfyUI': 8188,
    'Jupyter': 8888, 
    'Ollama': 11434,
    'Redis': 6379,
    'PostgreSQL': 5432
}

import socket
for service_name, port in common_ports.items():
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex(('localhost', port))
        sock.close()
        
        if result == 0:
            print(f"  ✅ {service_name} (:{port}) - Accessible")
        else:
            print(f"  🔴 {service_name} (:{port}) - Non accessible")
    except Exception:
        print(f"  ❓ {service_name} (:{port}) - Test échoué")

### Analyse du diagnostic Docker

Le diagnostic a verifie la disponibilite de Docker et l'acces aux services GenAI.

**Composantes verifiees** :

| Composante | Methode de verification | Resultat attendu |
|------------|------------------------|------------------|
| Docker CLI | Commande `docker --version` | Version Docker affichee |
| Docker Compose | Commande `docker compose version` | Version Compose affichee |
| Services actifs | Commande `docker ps` | Liste des conteneurs en cours |
| Connectivite | Test de ports (socket) | Services accessibles sur ports |

**Services GenAI et leurs ports** :
- ComfyUI : 8188 (interface web)
- Jupyter : 8888 (notebooks interactifs)
- Ollama : 11434 (LLM local)
- Redis : 6379 (cache)
- PostgreSQL : 5432 (base de donnees)

**Ressources systeme** (si `show_resources=True`) :
- CPU : Pourcentage d'utilisation
- RAM : Pourcentage et quantite utilisee
- Disque : Espace disponible pour les images Docker

> **Note technique** : Si Docker est disponible mais aucun service n'est actif, vous pouvez les demarrer avec `docker compose up -d` depuis le repertoire contenant le fichier docker-compose.genai.yml.

In [ ]:
# Interface interactive de gestion des services (widgets)
if not skip_widgets and notebook_mode == "interactive":
    try:
        import ipywidgets as widgets
        from IPython.display import clear_output, display
        
        print("🎛️  Interface interactive Docker Services")
        
        # Sélection service
        service_dropdown = widgets.Dropdown(
            options=managed_services + ['all'],
            value='all',
            description='Service:'
        )
        
        # Actions disponibles
        action_dropdown = widgets.Dropdown(
            options=['status', 'start', 'stop', 'restart', 'logs', 'health'],
            value='status',
            description='Action:'
        )
        
        # Boutons d'action
        execute_btn = widgets.Button(
            description='🚀 Exécuter',
            button_style='primary'
        )
        
        refresh_btn = widgets.Button(
            description='🔄 Actualiser',
            button_style='info'
        )
        
        # Zone de sortie
        output_area = widgets.Output(layout={'height': '300px', 'overflow': 'scroll'})
        
        def execute_action(b):
            with output_area:
                clear_output(wait=True)
                service = service_dropdown.value
                action = action_dropdown.value
                
                print(f"🔧 Exécution: {action} sur {service}")
                print(f"⏰ {datetime.now().strftime('%H:%M:%S')}")
                print("-" * 40)
                
                if action == 'status':
                    services_info = docker_manager.list_services()
                    if 'services' in services_info:
                        for svc in services_info['services']:
                            if service == 'all' or service in svc['name']:
                                print(f"📦 {svc['name']}: {svc['status']}")
                
                elif action == 'logs' and DOCKER_AVAILABLE:
                    if service != 'all':
                        cmd = ['docker', 'logs', '--tail', '20', service]
                        result = docker_manager.run_command(cmd, timeout=10)
                        if result['success']:
                            print(result['stdout'])
                        else:
                            print(f"❌ {result.get('error', 'Erreur logs')}")
                
                else:
                    print(f"⚠️  Action {action} non implémentée dans cette démo")
                    print("💡 Commandes Docker suggérées:")
                    if service != 'all':
                        print(f"   docker {action} {service}")
                    else:
                        print(f"   docker compose {action}")
        
        def refresh_services(b):
            # Actualiser la liste des services
            services_info = docker_manager.list_services()
            if 'services' in services_info:
                current_services = [s['name'].split('_')[0] for s in services_info['services']]
                service_dropdown.options = list(set(current_services + managed_services + ['all']))
        
        execute_btn.on_click(execute_action)
        refresh_btn.on_click(refresh_services)
        
        # Interface utilisateur
        controls = widgets.HBox([
            service_dropdown, 
            action_dropdown, 
            execute_btn, 
            refresh_btn
        ])
        
        ui = widgets.VBox([
            widgets.HTML("<h3>🐳 Gestionnaire Services Docker</h3>"),
            controls,
            output_area
        ])
        
        display(ui)
        
        # Exécution initiale du status
        execute_action(None)
        
    except ImportError:
        print("⚠️  Widgets non disponibles - Mode batch uniquement")
else:
    print("⏭️  Mode widgets désactivé")

In [ ]:
# Rapport final Docker Services
print("\n" + "="*60)
print("📋 RAPPORT DOCKER SERVICES MANAGEMENT")
print("="*60)

# Récapitulatif environnement
docker_available = docker_manager.get_docker_info().get('docker_available', False)
services_count = docker_manager.list_services().get('count', 0)

print(f"🐳 Docker Status: {'✅ Disponible' if docker_available else '❌ Non disponible'}")
print(f"📦 Services actifs: {services_count}")
print(f"⚙️  Services gérés: {', '.join(managed_services)}")

# Recommandations selon l'état
print("\n💡 RECOMMANDATIONS:")
if not docker_available:
    recommendations = [
        "Installer Docker Desktop depuis https://docker.com",
        "Vérifier que Docker Desktop est démarré",
        "Tester 'docker --version' en ligne de commande",
        "Configurer les ressources (RAM: 8GB min, CPU: 4 cores)"
    ]
else:
    if services_count == 0:
        recommendations = [
            "Lancer les services GenAI: 'docker compose -f docker-compose.genai.yml up -d'",
            "Vérifier les ports disponibles (8188, 8888, 11434)",
            "Attendre le démarrage complet (~2-5 minutes)",
            "Tester la connectivité avec les notebooks suivants"
        ]
    else:
        recommendations = [
            "Services fonctionnels - Passer aux notebooks d'images",
            "Monitorer l'usage RAM/CPU si problèmes de performance",
            "Configurer les logs Docker pour debug si nécessaire",
            "Sauvegarder la configuration Docker Compose"
        ]

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

# Prochaines étapes
print(f"\n🚀 PROCHAINES ÉTAPES:")
print(f"✅ Environment Setup (notebook 00-1) - Complété")
print(f"✅ Docker Services (notebook 00-2) - Complété")
print(f"➡️  API Endpoints Configuration (notebook 00-3) - À suivre")
print(f"➡️  Environment Validation (notebook 00-4) - À suivre")

# Export des variables pour autres notebooks
DOCKER_STATUS = {
    'available': docker_available,
    'services_count': services_count,
    'managed_services': managed_services,
    'timestamp': datetime.now().isoformat(),
    'notebook': '00-2-Docker-Services-Management'
}

print(f"\n📊 Variables exportées: DOCKER_STATUS")
print(f"🏁 Notebook Docker Services complété - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Interpretation du rapport Docker

Le rapport final synthetise l'etat de l'infrastructure Docker GenAI.

**Statut Docker observe** :
- Docker disponible : `DOCKER_STATUS['available']` (True/False)
- Services actifs : `DOCKER_STATUS['services_count']` (nombre)
- Services geres : comfyui, ollama, jupyter, redis, postgres

| Scenario | Interpretation | Action recommandee |
|----------|----------------|-------------------|
| Docker non disponible | Docker Desktop n'est pas installe ou non demarre | Installer/demarrer Docker Desktop |
| 0 services actifs | Les services GenAI ne sont pas demarres | Executer `docker compose up -d` |
| Services actifs | L'infrastructure fonctionne | Passer aux notebooks de generation |

**Variables exportees** :
- `DOCKER_STATUS` : Dictionnaire contenant le statut Docker pour les notebooks suivants

> **Note technique** : Les services Docker consomment des ressources systeme importantes. Sur une machine avec RAM limitee, demarrez uniquement les services necessaires.

## Conclusion du Notebook

### Resume des apprentissages

Ce notebook vous a permis de :
1. **Comprendre** l'architecture Docker GenAI et ses services
2. **Diagnostiquer** l'etat de Docker et des conteneurs
3. **Gerer** les services via la classe DockerServiceManager
4. **Interagir** avec les services via l'interface interactive (mode notebook)

### Services GenAI couverts

| Service | Port | Usage | GPU requis |
|---------|------|-------|------------|
| ComfyUI | 8188 | Generation d'images | Oui (20GB+) |
| Jupyter | 8888 | Notebooks interactifs | Optionnel |
| Ollama | 11434 | LLM local | Oui (8GB+) |
| Redis | 6379 | Cache | Non |
| PostgreSQL | 5432 | Base de donnees | Non |

### Prochaines etapes

1. Si Docker est disponible et fonctionne : Passez au notebook **00-3-API-Endpoints-Configuration**
2. Si Docker n'est pas installe : Installez Docker Desktop depuis https://docker.com
3. Si les services ne demarrent pas : Consultez le notebook **00-6-Local-Docker-Deployment**

> **Note technique** : La variable `DOCKER_STATUS` est exportee pour usage dans les notebooks suivants. Elle contient le statut de disponibilite Docker et le nombre de services actifs.